## Прогноз уровня сложности фильма для изучения английского языка

**Описание проекта**  

Просмотр фильмов на оригинальном языке - это популярный и действенный метод упражнений по изучению иностранных языков.  
Важно выбрать фильм, который подходит студенту по уровню сложности, чтобы студент понимал порядка 50-70 % диалогов. 

Требуется разработать модель для автоматического определения уровня сложности англоязычных фильмов.
Исходные данные: 
- Размеченный датасет с названиями фильмов в формате `excel`, субтитрами и меткой уровня сложности языка
- Файлы субтитров в формате `.srt`, отсортированные по каталогам в соответствии с уровнем сложности
- Словари Oxford по уровню сложности

In [1]:
# # Установка библиотек
#!pip3 install pysrt
#!pip3 install spacy
#!pip3 install pyprind

### Импорт библиотек и модулей

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
import pyprind
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# nltk.download('stopwords') # Загрузка после самого первого импорта библиотеки
# nltk.download('punkt') 
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, recall_score, precision_score
from sklearn.model_selection import GridSearchCV 
from time import time 

In [3]:
# Настроим широкоформатный Jupyter
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
# Настроим вывод всех столбцов DataFrame
pd.set_option('display.max_columns', None)

In [4]:
# Отключим предупреждения в jupyter notebook 
warnings.filterwarnings('ignore')

### Загрузка и предобработка `movies_labels.xlsx`

In [5]:
# загрузим данные из excel таблицы
df_m = pd.read_excel('C:/Users/svkb2/Desktop/Data Science/Film en_level/DataSet/movies_labels.xlsx', index_col='id')
df_m.head()

,Movie,Level
id,,
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+


In [6]:
# Список уникальных значений 'Level'
df_m['Level'].unique()

array(['B1', 'B2', 'A2/A2+', 'C1', 'B1, B2', 'A2/A2+, B1', 'A2'],
      dtype=object)

In [7]:
# Словарь новых уровней
new_level = {'A2': 'A2',
             'A2/A2+': 'A2',
             'B1': 'B1',
             'A2/A2+, B1': 'B1',
             'B2': 'B2',
             'B1, B2': 'B2',
             'C1': 'C1'}
# Заменим значения в колонке 'Level' на новые
df_m['Level'] = df_m['Level'].map(new_level)

In [8]:
df_m['Level'].unique()

array(['B1', 'B2', 'A2', 'C1'], dtype=object)

In [9]:
# Проверим данные на наличие дубликатов
print('Кол-во дубликатов:', df_m.duplicated().sum())
print()
df_m[df_m.duplicated(keep=False)]

Кол-во дубликатов: 3



,Movie,Level
id,,
38,Powder(1995),B1
43,Inside_out(2015),B1
44,Inside_out(2015),B1
68,Powder(1995),B1
83,The_terminal(2004),B1
99,The_terminal(2004),B1


In [10]:
# Удалим дубликаты
df_m = df_m.drop_duplicates().reset_index(drop = True)
df_m.duplicated().sum()

0

In [11]:
df_m['Movie'] = df_m['Movie'] + '.srt'
df_m.head()

,Movie,Level
0,10_Cloverfield_lane(2016).srt,B1
1,10_things_I_hate_about_you(1999).srt,B1
2,A_knights_tale(2001).srt,B2
3,A_star_is_born(2018).srt,B2
4,Aladdin(1992).srt,A2


### Загрузка и предобработка `Oxford_words.xlsx`

Предварительно слова из файлов формата pdf были перенесены в файл формата xlsx.

In [12]:
# загрузим данные из excel таблицы
df_o = pd.read_excel('C:/Users/svkb2/Desktop/Data Science/Film en_level/DataSet/Oxford_words.xlsx')
df_o

,Level,Words
0,A2,a an about above across action activity actor ...
1,B1,absolutely academic access accommodation accou...
2,B2,abandon absolute acceptable accompany accurate...
3,C1,abolish abortion absence absent absurd abundan...


### Загрузка и предобработка субтитров

In [13]:
# Каталог с данными
basepath = 'C:/Users/svkb2/Desktop/Data Science/Film en_level/DataSet/'
films_name = []
for f in ('A2', 'B1', 'B2', 'C1', 'Subtitles'):
    films_name = films_name + os.listdir(path=os.path.join(basepath, f))
# Выведем общее кол-во субтитров
print(f'Общее кол-во субтитров: {len(films_name)}')
# Выведем список названий файлов с субтитрами
print('\nCписок названий файлов с субтитрами:\n')
print(films_name[:9])

Общее кол-во субтитров: 284

Cписок названий файлов с субтитрами:

['The Walking Dead-S01E01-Days Gone Bye.English.srt', 'The Walking Dead-S01E02-Guts.English.srt', 'The Walking Dead-S01E03-Tell It To The Frogs.English.srt', 'The Walking Dead-S01E04-Vatos.English.srt', 'The Walking Dead-S01E05-Wildfire.English.srt', 'The Walking Dead-S01E06-TS-19.English.srt', 'AmericanBeauty1999.BRRip.srt', 'Angelas.Christmas.Wish.2020.srt', 'Indiana Jones And The Last Crusade DVDRip Xvid -IZON-.srt']


In [14]:
df_s = pd.DataFrame()
for f in ('A2', 'B1', 'B2', 'C1', 'Subtitles'):
    path = os.path.join(basepath, f)
    for file in os.listdir(path):
        try:
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
        except:
            with open(os.path.join(path, file), 'r', encoding='iso-8859-1') as infile:
                txt = infile.read()
        df_s = df_s.append([[file, txt, f]], ignore_index=True)
df_s.columns = ['File', 'Subtitles', 'Level']


In [15]:
df_s

,File,Subtitles,Level
0,The Walking Dead-S01E01-Days Gone Bye.English.srt,"1\n00:00:03,169 --> 00:00:05,171\n( bugs chitt...",A2
1,The Walking Dead-S01E02-Guts.English.srt,"1\n00:00:03,045 --> 00:00:05,047\n- ( birds ch...",A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,"1\n00:00:03,003 --> 00:00:04,671\n( thunder ru...",A2
3,The Walking Dead-S01E04-Vatos.English.srt,"1\n00:00:03,045 --> 00:00:05,422\n( birds chir...",A2
4,The Walking Dead-S01E05-Wildfire.English.srt,"1\n00:00:03,420 --> 00:00:04,922\n- ( walkie-t...",A2
...,...,...,...
279,Warm_bodies(2013).srt,"2\n00:00:26,559 --> 00:00:28,627\n<i>What am I...",Subtitles
280,Westworld_scenes_of_Dr_Robert_Ford.srt,"1\n00:00:00,000 --> 00:00:21,179\n[Music]\n\n2...",Subtitles
281,We_are_the_Millers(2013).srt,"1\n00:00:02,400 --> 00:00:03,731\n<i>Oh, my Go...",Subtitles
282,While_You_Were_Sleeping(1995).srt,"1\n00:02:20,760 --> 00:02:24,720\nLUCY: <i>Oka...",Subtitles


In [16]:
df_s['Subtitles'][0][:500]

'1\n00:00:03,169 --> 00:00:05,171\n( bugs chittering )\n\n2\n00:00:26,859 --> 00:00:28,861\n( brakes squeak )\n\n3\n00:00:30,696 --> 00:00:34,158\n- ( engine stops )\n- ( trunk clicks )\n\n4\n00:00:37,954 --> 00:00:39,956\n( bird cawing )\n\n5\n00:01:33,551 --> 00:01:35,553\n( birds chirping )\n\n6\n00:02:12,381 --> 00:02:14,383\n( flies buzzing )\n\n7\n00:02:19,639 --> 00:02:21,641\n( metal rattling )\n\n8\n00:02:45,790 --> 00:02:47,792\n( shuffling footsteps )\n\n9\n00:03:29,458 --> 00:03:31,210\nLittle girl?\n\n10\n00:03:32,461 --'

In [17]:
print('Кол-во субтитров с заданным уровнем английского:   ', df_s.query('Level != "Subtitles"')['Level'].count())
print('Кол-во субтитров, где уровень английского не задан:', df_s.query('Level == "Subtitles"')['Level'].count())

Кол-во субтитров с заданным уровнем английского:    163
Кол-во субтитров, где уровень английского не задан: 121


In [18]:
for i in range(len(df_s['File'])):
    if df_s['Level'][i] == 'Subtitles':
        try:
            df_s['Level'][i] = df_m.loc[df_m['Movie']==df_s['File'][i]]['Level'].values[0]
        except:
            df_s['Level'][i] = np.nan

In [19]:
df_s

,File,Subtitles,Level
0,The Walking Dead-S01E01-Days Gone Bye.English.srt,"1\n00:00:03,169 --> 00:00:05,171\n( bugs chitt...",A2
1,The Walking Dead-S01E02-Guts.English.srt,"1\n00:00:03,045 --> 00:00:05,047\n- ( birds ch...",A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,"1\n00:00:03,003 --> 00:00:04,671\n( thunder ru...",A2
3,The Walking Dead-S01E04-Vatos.English.srt,"1\n00:00:03,045 --> 00:00:05,422\n( birds chir...",A2
4,The Walking Dead-S01E05-Wildfire.English.srt,"1\n00:00:03,420 --> 00:00:04,922\n- ( walkie-t...",A2
...,...,...,...
279,Warm_bodies(2013).srt,"2\n00:00:26,559 --> 00:00:28,627\n<i>What am I...",B1
280,Westworld_scenes_of_Dr_Robert_Ford.srt,"1\n00:00:00,000 --> 00:00:21,179\n[Music]\n\n2...",NaN
281,We_are_the_Millers(2013).srt,"1\n00:00:02,400 --> 00:00:03,731\n<i>Oh, my Go...",B1
282,While_You_Were_Sleeping(1995).srt,"1\n00:02:20,760 --> 00:02:24,720\nLUCY: <i>Oka...",B1


In [20]:
df_s.query('Level.isna()')

,File,Subtitles,Level
179,Breaking_Bad_The_Movie(2017).srt,"1\n00:00:05,299 --> 00:00:08,220\nwhat hi good...",NaN
180,Bren╨Т.Brown.The.Call.to.Courage.2019.720.NF.7...,"﻿1\n00:00:07,216 --> 00:00:09,676\n[presenter]...",NaN
185,Casper.srt,"1\n00:00:18,040 --> 00:00:21,870\n[Music]\n\n2...",NaN
204,Gogo_Loves_English.srt,"1\n00:00:01,130 --> 00:00:18,359\n[Music]\n\n2...",NaN
207,Harry_Potter_and_the_philosophers_stone(2001).srt,"1\n00:01:22,065 --> 00:01:27,070\nI should've ...",NaN
238,Pride_and_Prejudice.srt,"1\n00:00:20,020 --> 00:00:42,009\n[Music]\n\n2...",NaN
253,The_Ghost_Writer.srt,"1\n00:00:04,670 --> 00:00:18,579\n[Music]\n\n2...",NaN
280,Westworld_scenes_of_Dr_Robert_Ford.srt,"1\n00:00:00,000 --> 00:00:21,179\n[Music]\n\n2...",NaN


In [21]:
df_s = df_s.dropna(subset=['Level']).reset_index(drop = True)

### Очистка текста

In [22]:
# Регулярнеы выражения для очистки текста
HTML = r'<.*?>' # html тэги меняем на пробел
TAG = r'{.*?}' # тэги меняем на пробел
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]' # комменты в скобках меняем на пробел
LETTERS = r'[^a-zA-Z\.,!? ]' # все что не буквы меняем на пробел 
SPACES = r'([ ])\1+' # повторяющиеся пробелы меняем на один пробел
DOTS = r'[\.]+' # многоточие меняем на точку
SYMB = r"[^\w\d'\s]" # знаки препинания кроме апострофа

In [23]:
# Функция очистки текста
def clean_subtitles(txt):
    txt = re.sub(HTML, ' ', txt) # html тэги меняем на пробел
    txt = re.sub(COMMENTS, ' ', txt) # комменты в скобках меняем на пробел
    txt = re.sub(LETTERS, ' ', txt) # все что не буквы меняем на пробел
    txt = re.sub(DOTS, r'.', txt) # многоточие меняем на точку
    txt = re.sub(SYMB, '', txt) # знаки препинания кроме апострофа на пустую строку
    txt = re.sub('www', '', txt) # кое-где остаётся www, то же меняем на пустую строку
    txt = txt.lstrip() # обрезка пробелов слева
    txt = txt.encode('ascii', 'ignore').decode() # удаляем все что не ascii символы   
    txt = txt.lower() # текст в нижний регистр
    txt = re.sub(SPACES, r'\1', txt) # повторяющиеся пробелы меняем на один пробел
    return txt

In [24]:
df_s['Subtitles'] = df_s['Subtitles'].apply(lambda x: clean_subtitles(x))
df_o['Words'] = df_o['Words'].apply(lambda x: clean_subtitles(x))

In [25]:
display(df_s.head())
display(df_o)

,File,Subtitles,Level
0,The Walking Dead-S01E01-Days Gone Bye.English.srt,little girl i m a policeman little girl don t ...,A2
1,The Walking Dead-S01E02-Guts.English.srt,boy mom right here any luck how do we tell if ...,A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,merle that s right you heard me bitch you got ...,A2
3,The Walking Dead-S01E04-Vatos.English.srt,what nothing it s not nothing it s always some...,A2
4,The Walking Dead-S01E05-Wildfire.English.srt,walkie talkie squawks rick morgan i don t know...,A2


,Level,Words
0,A2,a an about above across action activity actor ...
1,B1,absolutely academic access accommodation accou...
2,B2,abandon absolute acceptable accompany accurate...
3,C1,abolish abortion absence absent absurd abundan...


In [26]:
#n = 1000000
#bar = pyprind.ProgBar(n)
#for i in range(n):
#    bar.update()

### Разделение на слова

In [27]:
# импортируем метод word_tokenize
from nltk.tokenize import word_tokenize
df_s['Subtitles'] = df_s['Subtitles'].apply(lambda x: word_tokenize(x))
df_o['Words'] = df_o['Words'].apply(lambda x: word_tokenize(x))
display(df_s.head())
display(df_o)

,File,Subtitles,Level
0,The Walking Dead-S01E01-Days Gone Bye.English.srt,"[little, girl, i, m, a, policeman, little, gir...",A2
1,The Walking Dead-S01E02-Guts.English.srt,"[boy, mom, right, here, any, luck, how, do, we...",A2
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,"[merle, that, s, right, you, heard, me, bitch,...",A2
3,The Walking Dead-S01E04-Vatos.English.srt,"[what, nothing, it, s, not, nothing, it, s, al...",A2
4,The Walking Dead-S01E05-Wildfire.English.srt,"[walkie, talkie, squawks, rick, morgan, i, don...",A2


,Level,Words
0,A2,"[a, an, about, above, across, action, activity..."
1,B1,"[absolutely, academic, access, accommodation, ..."
2,B2,"[abandon, absolute, acceptable, accompany, acc..."
3,C1,"[abolish, abortion, absence, absent, absurd, a..."


In [28]:
all = []
for i in range(len(df_o['Words'])):
    all = all + df_o['Words'][i]
df_o.loc[len(df_o.index)] = ['All', all] 

In [29]:
display(df_o)

,Level,Words
0,A2,"[a, an, about, above, across, action, activity..."
1,B1,"[absolutely, academic, access, accommodation, ..."
2,B2,"[abandon, absolute, acceptable, accompany, acc..."
3,C1,"[abolish, abortion, absence, absent, absurd, a..."
4,All,"[a, an, about, above, across, action, activity..."


### Удаление стоп-слов

In [30]:
unique_stops = set(stopwords.words('english')) # Устанавливаем множество english стоп-слов

In [31]:
# Функция удаления стоп-слов
def del_stopwords(tokens):
    no_stops = [] 
    for token in tokens:
        if token not in unique_stops:
            no_stops.append(token)
    return no_stops

In [32]:
df_s['Subtitles'] = df_s['Subtitles'].apply(lambda x: del_stopwords(x))
df_o['Words'] = df_o['Words'].apply(lambda x: del_stopwords(x))

### Лемматизация

In [33]:
# Cоздаём объект этого класса
lemmatizer = WordNetLemmatizer()

In [34]:
# Функция лематизации
def lemma(tokens):
    lemmatized = []
    for token in tokens:
        token = lemmatizer.lemmatize(token)
        lemmatized.append(token)
    return lemmatized

In [35]:
df_s['Subtitles'] = df_s['Subtitles'].apply(lambda x: lemma(x))
df_o['Words'] = df_o['Words'].apply(lambda x: lemma(x))

### Оставим в субтитрах только слова из словаря Oxford

После очистки текста и удаления стоп-слов в субтитрах осталось много мусора.  
Чтобы избавиться от него, оставим в тексте только слова из "словаря" Oxford.  

In [36]:
# Функция отбора слов из словаря Oxford
def oxford_words(tokens, level):
    oxford = []
    oxford_list = df_o.loc[df_o['Level']==level]['Words'].values[0]
#    for token in tokens:
#        if token in oxford_list:
#            oxford.append(token)
    oxford = [token for token in tokens if token in oxford_list]
    return oxford

In [37]:
df_s['Subtitles'] = df_s['Subtitles'].apply(lambda x: oxford_words(x, 'All'))

### Создание дополнительных признаков

- `total_words` - общее кол-во слов в субтитрах
- `unique_words` - кол-во уникальных слов в субтитрах

In [38]:
for i in range(len(df_s['Subtitles'])):
    df_s.loc[i, 'total_words'] = len(df_s['Subtitles'][i])
    df_s.loc[i, 'unique_words'] = len(set(df_s['Subtitles'][i]))

In [39]:
# Функция подсчета уникального количества слов уровня
def oxford_count(tokens, level):
    oxford = []
    oxford_list = df_o.loc[df_o['Level']==level]['Words'].values[0]
    oxford = [token for token in tokens if token in oxford_list]
    return len(set(oxford))

- `A2` - кол-во слов в субтитрах уровня сложности Oxford A2
- `B1` - кол-во слов в субтитрах уровня сложности Oxford B1
- `B2` - кол-во слов в субтитрах уровня сложности Oxford B2
- `C1` - кол-во слов в субтитрах уровня сложности Oxford C1

In [40]:
for i in ['A2', 'B1', 'B2', 'C1']:
    df_s[i] = df_s['Subtitles'].apply(lambda x: oxford_count(x, i))

In [41]:
display(df_s.describe().T)

,count,mean,std,min,25%,50%,75%,max
total_words,276.0,2357.025362,1088.059198,11.0,1665.00,2042.5,3094.50,5734.0
unique_words,276.0,655.119565,253.307496,1.0,571.75,646.0,808.00,1203.0
A2,276.0,456.916667,166.168513,1.0,407.75,448.0,562.25,775.0
B1,276.0,80.594203,36.310607,0.0,62.00,79.0,100.00,179.0
B2,276.0,86.710145,41.054147,0.0,67.75,85.5,108.25,219.0
C1,276.0,45.695652,22.114778,0.0,34.00,46.0,57.00,115.0


### Удаление аномальных данных из списка субтитров

Из описания числовых колонок выше видно, что некоторые субтитры либо очень короткие, либо представлены небольшим кол-вом уникальных слов из словаря Oxford.  
Выведем те, где кол-во уникальных слов словаря меньше 100.

In [42]:
display(df_s.query('unique_words < 100'))

,File,Subtitles,Level,total_words,unique_words,A2,B1,B2,C1
45,"Crown, The S01E03 - Windsor.en.FORCED.srt","[dear, aunt, forgive, come, directly, oh, dear...",B2,11.0,10.0,7,1,2,0
50,"Crown, The S01E05 - Smoke and Mirrors.en.FORCE...","[time, history, queen, crown, big, day, public...",B2,21.0,20.0,15,3,2,3
55,"Crown, The S01E07 - Scientia Potentia Est.en.F...","[always, begin, meal, absolutely, begin, take,...",B2,18.0,15.0,13,2,0,0
62,"Crown, The S01E10 - Gloriana.en.FORCED.srt","[delighted, meet, continue, discussion, fundin...",B2,47.0,41.0,33,1,7,4
137,Suits S04E01 EngSub.srt,"[u, u, u, u, u, u, u, u, u, u, u, u, u, u, u, ...",C1,1059.0,1.0,1,0,0,0
138,Suits S04E02 EngSub.srt,"[u, u, u, u, u, u, u, u, u, u, u, u, u, u, u, ...",C1,1080.0,1.0,1,0,0,0
139,Suits S04E03 EngSub.srt,"[u, u, u, u, u, u, u, u, u, u, u, u, u, u, u, ...",C1,1007.0,1.0,1,0,0,0
140,Suits S04E04 EngSub.srt,"[u, u, u, u, u, u, u, u, u, u, u, u, u, u, u, ...",C1,1109.0,1.0,1,0,0,0
141,Suits S04E05 EngSub.srt,"[u, u, u, u, u, u, u, u, u, u, u, u, u, u, u, ...",C1,1045.0,1.0,1,0,0,0
142,Suits S04E06 EngSub.srt,"[u, u, u, u, u, u, u, u, u, u, u, u, u, u, u, ...",C1,1003.0,1.0,1,0,0,0


Удалим их.

In [43]:
df_s = df_s.query('unique_words >= 100').reset_index(drop = True)
display(df_s.describe().T)
print('Размер датафрейма субтитров:', df_s.shape)

,count,mean,std,min,25%,50%,75%,max
total_words,256.0,2474.082031,1035.814408,706.0,1836.00,2085.0,3147.75,5734.0
unique_words,256.0,705.902344,182.924849,284.0,585.75,657.0,814.25,1203.0
A2,256.0,492.285156,111.545510,228.0,420.00,459.5,568.00,775.0
B1,256.0,86.863281,29.619827,22.0,66.00,81.0,103.00,179.0
B2,256.0,93.441406,34.492120,23.0,72.00,87.0,111.00,219.0
C1,256.0,49.238281,18.800549,6.0,36.00,47.0,58.00,115.0


Размер датафрейма субтитров: (256, 9)


### Сборка текста из токенов

In [44]:
df_s['Subtitles'] = df_s['Subtitles'].apply(lambda x:TreebankWordDetokenizer().detokenize(x))
display(df_s)

,File,Subtitles,Level,total_words,unique_words,A2,B1,B2,C1
0,The Walking Dead-S01E01-Days Gone Bye.English.srt,little girl policeman little girl afraid littl...,A2,1180.0,426.0,326,37,53,23
1,The Walking Dead-S01E02-Guts.English.srt,boy right luck tell poison one sure way know a...,A2,1103.0,399.0,314,43,35,18
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,right problem bring man enough take chain take...,A2,1317.0,446.0,353,36,45,26
3,The Walking Dead-S01E04-Vatos.English.srt,nothing nothing always something dad teach tie...,A2,1196.0,429.0,330,46,46,19
4,The Walking Dead-S01E05-Wildfire.English.srt,know know hear maybe right hope found family b...,A2,1043.0,420.0,315,42,49,27
...,...,...,...,...,...,...,...,...,...
251,Venom(2018).srt,life foundation control specimen home go copy ...,B2,2762.0,802.0,528,90,145,56
252,Warm_bodies(2013).srt,life pale get eat better terrible stand straig...,B1,1612.0,485.0,372,49,49,27
253,We_are_the_Millers(2013).srt,oh god full double way across sky oh god oh go...,B1,5144.0,1034.0,720,119,148,66
254,While_You_Were_Sleeping(1995).srt,two thing remember childhood remember orange f...,B1,3365.0,770.0,576,90,88,34


### Кодирование целевого признака 

In [45]:
# Кодирование целевого признака 
code_level = {'A2': 1,
             'B1': 2,
             'B2': 3,
             'C1': 4}
# Заменим значения в колонке 'Level' на новые
df_s['Level'] = df_s['Level'].map(code_level)

### Дисбаланс классов

In [46]:
# Rоличество субтитров по уровням сложности
df_s['Level'].value_counts()

3    144
2     57
1     32
4     23
Name: Level, dtype: int64

В данных присутствует дисбаланс целевого признака:
- уровень сложности 3 (B2) - 144  
- уровень сложности 2 (B1) - 57
- уровень сложности 4 (C1) - 32
- уровень сложности 1 (A2) - 23

### Разбивка данных на обучающую и тестовую выборки

Произведем разбивку данных на обучающую и тестовую выборки в соотношени 75/25 %.  
При этом, важно сохранить соотношение балансов классов в обеих выборках.  
Поэтому будем использовать параметр `'stratify'`. 

**Выделим целевой признак и предварительные признаки для кроссвалидации**

In [47]:
# Целевой признак
target = df_s['Level']
# Признаки
features = df_s.drop(['File', 'Level'], axis=1)
print('\nРазмер target:  ', target.shape, '\nРазмер features:', features.shape)


Размер target:   (256,) 
Размер features: (256, 7)


**Разобъем данные на тренировочную и тестовую выборки**

In [48]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)
print('\nРазмер features_train:', features_train.shape, '\nРазмер target_train  :', target_train.shape)
print('\nРазмер features_test : ', features_test.shape, '\nРазмер target_test   : ', target_test.shape)
print(f'\nДоля тренировочной выборки : {target_train.shape[0]/target.shape[0]:.0%}')
print(f'Доля тестовой выборки      : {target_test.shape[0]/target.shape[0]:.0%}')


Размер features_train: (192, 7) 
Размер target_train  : (192,)

Размер features_test :  (64, 7) 
Размер target_test   :  (64,)

Доля тренировочной выборки : 75%
Доля тестовой выборки      : 25%


### Перевод субтитров в числовые векторы

In [49]:
vectorizer = CountVectorizer(analyzer = "word", 
                             lowercase = True, 
                             tokenizer = None, 
                             preprocessor = None, 
                             stop_words = None, 
                             max_features = None)

In [50]:
subtitles_cv_train = vectorizer.fit_transform(features_train['Subtitles'])
subtitles_cv_test = vectorizer.transform(features_test['Subtitles'])

In [51]:
print('\nРазмер обучающей матрицы субтитров:', subtitles_cv_train.shape)
print('Размер тестовой матрицы субтитров :', subtitles_cv_test.shape)


Размер обучающей матрицы субтитров: (192, 4579)
Размер тестовой матрицы субтитров : (64, 4579)


In [52]:
tokens = vectorizer.get_feature_names_out() # получим список токенов
subtitles_cv_train_df = pd.DataFrame(data = subtitles_cv_train.toarray(), index = features_train.index, columns = tokens)
subtitles_cv_test_df = pd.DataFrame(data = subtitles_cv_test.toarray(), index = features_test.index, columns = tokens)
display(subtitles_cv_train_df.head())
display(subtitles_cv_test_df.head())

abandon  ability  able  abortion  abroad  absence  absent  absolute  \
240        0        0     1         0       1        0       0         0   
49         0        2     2         0       0        0       0         0   
19         0        0     5         0       0        0       0         0   
219        0        0     0         0       0        0       0         0   
100        0        0     0         0       0        0       0         0   

     absolutely  abstract  absurd  abuse  academic  academy  accelerate  \
240           3         0       0      0         0        0           0   
49            0         0       0      0         0        0           0   
19            0         0       0      0         0        0           0   
219           0         0       0      0         0        0           0   
100           0         0       0      0         0        0           0   

     accent  accept  acceptable  acceptance  access  accessible  accident  \
240       0       0           0           0       0           0         2   
49        0       0           0           0       0           0         0   
19        0       1           0           1       4           1         0   
219       0       0           0           0       0           0         0   
100       1       0           0           1       0           0         1   

     accidentally  accommodate  accommodation  accompany  accomplish  \
240             0            0              0          0           1   
49              0            0              0          0           0   
19              0            0              0          0           1   
219             0            0              0          0           0   
100             0            0              0          0           0   

     accomplishment  according  accordingly  account  accountable  accountant  \
240               0          1            0        2            0           0   
49                0          0            0        0            1           0   
19                0          0            0        0            0           1   
219               0          0            0        0            0           0   
100               0          0            0        2            0           0   

     accumulate  accuracy  accurate  accurately  accusation  accuse  accused  \
240           0         0         0           0           0       0        1   
49            0         0         0           0           0       0        0   
19            0         0         0           0           0       0        1   
219           0         0         0           0           0       0        0   
100           0         0         0           0           0       0        0   

     achieve  achievement  acid  acknowledge  acquire  acquisition  acre  \
240        0            0     0            0        0            0     0   
49         0            0     0            1        0            0     1   
19         0            1     0            0        0            0     0   
219        0            0     0            0        0            0     0   
100        0            0     0            0        0            0     0   

     across  act  action  activate  activation  active  activist  activity  \
240       1    0       1         0           0       0         0         0   
49        0    0       0         0           0       0         0         0   
19        0    0       0         0           0       0         0         0   
219       0    1       0         0           0       0         0         0   
100       0    0       4         0           0       0         0         0   

     actor  actress  actual  actually  acute  ad  adapt  adaptation  add  \
240      0        0       0         0      0   0      1           0    2   
49       0        0       0         0      0   0      1           0    0   
19       0        0       0         7      0   0      0           0    1   
219

abandon  ability  able  abortion  abroad  absence  absent  absolute  \
71         0        0     5         0       0        0       0         0   
191        0        0     0         0       0        0       0         0   
160        0        0     0         0       0        0       1         0   
99         0        0     0         0       0        0       0         0   
144        0        0     1         0       0        0       0         0   

     absolutely  abstract  absurd  abuse  academic  academy  accelerate  \
71            1         0       0      1         0        0           0   
191           2         0       0      0         0        0           0   
160           4         0       0      0         0        0           0   
99            0         0       0      0         0        0           0   
144           1         0       0      0         0        0           0   

     accent  accept  acceptable  acceptance  access  accessible  accident  \
71        0       0           0           0       3           0         0   
191       0       0           0           0       0           0         0   
160       0       1           0           0       0           0         0   
99        0       0           0           0       0           0         2   
144       1       0           0           0       0           0         0   

     accidentally  accommodate  accommodation  accompany  accomplish  \
71              0            0              0          0           0   
191             0            0              0          0           0   
160             0            0              0          0           0   
99              0            0              0          0           0   
144             0            0              0          0           0   

     accomplishment  according  accordingly  account  accountable  accountant  \
71                0          0            0        0            0           0   
191               0          0            0        0            0           0   
160               0          0            0        0            0           0   
99                0          0            0        0            0           0   
144               0          0            0        0            0           0   

     accumulate  accuracy  accurate  accurately  accusation  accuse  accused  \
71            0         0         0           0           0       0        3   
191           0         0         0           0           0       0        0   
160           0         0         0           0           0       0        0   
99            0         0         0           0           2       1        2   
144           0         0         0           0           0       0        0   

     achieve  achievement  acid  acknowledge  acquire  acquisition  acre  \
71         0            0     0            0        0            0     0   
191        0            0     0            0        0            0     0   
160        0            0     0            0        0            0     0   
99         0            0     0            0        0            0     0   
144        0            0     0            0        0            0     0   

     across  act  action  activate  activation  active  activist  activity  \
71        0    0       1         0           0       0         0         0   
191       2    1       0         0           0       0         0         0   
160       0    0       1         0           0       0         0         0   
99        0    0       1         0           0       0         0         0   
144       1    1       1         0           0       0         0         0   

     actor  actress  actual  actually  acute  ad  adapt  adaptation  add  \
71       0        0       0         3      0   0      0           0    0   
191      0        0       0         2      0   0      0           0    0   
160      0        0       0        16      0   1      0           0    1   
99 

### Итоговое формирование обучающих и тестовых признаков 

In [53]:
features_train = features_train.drop('Subtitles', axis=1)
features_test = features_test.drop('Subtitles', axis=1)
features_train = pd.concat([subtitles_cv_train_df, features_train], axis = 1)
features_test = pd.concat([subtitles_cv_test_df, features_test], axis = 1)
print('\nРазмер features_train:', features_train.shape, '\nРазмер target_train  :', target_train.shape)
print('\nРазмер features_test : ', features_test.shape, '\nРазмер target_test   : ', target_test.shape)


Размер features_train: (192, 4585) 
Размер target_train  : (192,)

Размер features_test :  (64, 4585) 
Размер target_test   :  (64,)


## Обучение моделей

Создадим функцию обучения моделей, где подбор гиперпараметров производит GridSearchCV.

In [54]:
def model_training(model, features_train, target_train, parameters):
    model_grid = GridSearchCV(model, parameters, cv=4, scoring='f1_weighted', verbose=5)
    model_grid.fit(features_train, target_train)
    return model_grid.best_score_, model_grid.best_params_

### Дерево решений

In [55]:
model = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
parameters = {'max_depth': [None] + [i for i in range(2, 7)], 'min_samples_split': range(4,21,2)}

In [56]:
f1, best_parameters = model_training(model, features_train, target_train, parameters)
print('\nЛучшие гиперпараметры модели:', best_parameters)
print('F1_CV модели:            ', f1)

Fitting 4 folds for each of 54 candidates, totalling 216 fits
[CV 1/4] END max_depth=None, min_samples_split=4;, score=0.552 total time=   0.0s
[CV 2/4] END max_depth=None, min_samples_split=4;, score=0.517 total time=   0.0s
[CV 3/4] END max_depth=None, min_samples_split=4;, score=0.589 total time=   0.0s
[CV 4/4] END max_depth=None, min_samples_split=4;, score=0.574 total time=   0.0s
[CV 1/4] END max_depth=None, min_samples_split=6;, score=0.534 total time=   0.0s
[CV 2/4] END max_depth=None, min_samples_split=6;, score=0.535 total time=   0.0s
[CV 3/4] END max_depth=None, min_samples_split=6;, score=0.542 total time=   0.0s
[CV 4/4] END max_depth=None, min_samples_split=6;, score=0.572 total time=   0.0s
[CV 1/4] END max_depth=None, min_samples_split=8;, score=0.540 total time=   0.0s
[CV 2/4] END max_depth=None, min_samples_split=8;, score=0.512 total time=   0.0s
[CV 3/4] END max_depth=None, min_samples_split=8;, score=0.574 total time=   0.0s
[CV 4/4] END max_depth=None, min_sam

[CV 2/4] END .max_depth=3, min_samples_split=18;, score=0.634 total time=   0.0s
[CV 3/4] END .max_depth=3, min_samples_split=18;, score=0.519 total time=   0.0s
[CV 4/4] END .max_depth=3, min_samples_split=18;, score=0.499 total time=   0.0s
[CV 1/4] END .max_depth=3, min_samples_split=20;, score=0.385 total time=   0.0s
[CV 2/4] END .max_depth=3, min_samples_split=20;, score=0.634 total time=   0.0s
[CV 3/4] END .max_depth=3, min_samples_split=20;, score=0.519 total time=   0.0s
[CV 4/4] END .max_depth=3, min_samples_split=20;, score=0.499 total time=   0.0s
[CV 1/4] END ..max_depth=4, min_samples_split=4;, score=0.509 total time=   0.0s
[CV 2/4] END ..max_depth=4, min_samples_split=4;, score=0.562 total time=   0.0s
[CV 3/4] END ..max_depth=4, min_samples_split=4;, score=0.583 total time=   0.0s
[CV 4/4] END ..max_depth=4, min_samples_split=4;, score=0.547 total time=   0.0s
[CV 1/4] END ..max_depth=4, min_samples_split=6;, score=0.509 total time=   0.0s
[CV 2/4] END ..max_depth=4, 

[CV 1/4] END .max_depth=6, min_samples_split=16;, score=0.502 total time=   0.0s
[CV 2/4] END .max_depth=6, min_samples_split=16;, score=0.548 total time=   0.0s
[CV 3/4] END .max_depth=6, min_samples_split=16;, score=0.476 total time=   0.0s
[CV 4/4] END .max_depth=6, min_samples_split=16;, score=0.584 total time=   0.0s
[CV 1/4] END .max_depth=6, min_samples_split=18;, score=0.502 total time=   0.0s
[CV 2/4] END .max_depth=6, min_samples_split=18;, score=0.548 total time=   0.0s
[CV 3/4] END .max_depth=6, min_samples_split=18;, score=0.476 total time=   0.0s
[CV 4/4] END .max_depth=6, min_samples_split=18;, score=0.563 total time=   0.0s
[CV 1/4] END .max_depth=6, min_samples_split=20;, score=0.525 total time=   0.0s
[CV 2/4] END .max_depth=6, min_samples_split=20;, score=0.548 total time=   0.0s
[CV 3/4] END .max_depth=6, min_samples_split=20;, score=0.476 total time=   0.0s
[CV 4/4] END .max_depth=6, min_samples_split=20;, score=0.481 total time=   0.0s

Лучшие гиперпараметры модел

Занесем данные в таблицу `results`

In [57]:
results = pd.DataFrame ({'model':['DecisionTreeClassifier'], 'best_parameters_train':[best_parameters], 'F1_CV':[f1]})

### Случайный лес

In [58]:
model = RandomForestClassifier(random_state=12345, class_weight='balanced')
parameters = {'n_estimators': range(50, 101, 5), 'max_depth': [None] + [i for i in range(2, 7)]}

In [59]:
f1, best_parameters = model_training(model, features_train, target_train, parameters)
print('\nЛучшие гиперпараметры модели:', best_parameters)
print('F1_CV модели:            ', f1)

Fitting 4 folds for each of 66 candidates, totalling 264 fits
[CV 1/4] END ...max_depth=None, n_estimators=50;, score=0.537 total time=   0.0s
[CV 2/4] END ...max_depth=None, n_estimators=50;, score=0.523 total time=   0.0s
[CV 3/4] END ...max_depth=None, n_estimators=50;, score=0.492 total time=   0.0s
[CV 4/4] END ...max_depth=None, n_estimators=50;, score=0.576 total time=   0.0s
[CV 1/4] END ...max_depth=None, n_estimators=55;, score=0.455 total time=   0.0s
[CV 2/4] END ...max_depth=None, n_estimators=55;, score=0.520 total time=   0.0s
[CV 3/4] END ...max_depth=None, n_estimators=55;, score=0.496 total time=   0.0s
[CV 4/4] END ...max_depth=None, n_estimators=55;, score=0.539 total time=   0.0s
[CV 1/4] END ...max_depth=None, n_estimators=60;, score=0.537 total time=   0.0s
[CV 2/4] END ...max_depth=None, n_estimators=60;, score=0.485 total time=   0.0s
[CV 3/4] END ...max_depth=None, n_estimators=60;, score=0.496 total time=   0.0s
[CV 4/4] END ...max_depth=None, n_estimators=60

[CV 3/4] END ......max_depth=3, n_estimators=65;, score=0.527 total time=   0.0s
[CV 4/4] END ......max_depth=3, n_estimators=65;, score=0.548 total time=   0.0s
[CV 1/4] END ......max_depth=3, n_estimators=70;, score=0.474 total time=   0.0s
[CV 2/4] END ......max_depth=3, n_estimators=70;, score=0.621 total time=   0.0s
[CV 3/4] END ......max_depth=3, n_estimators=70;, score=0.536 total time=   0.0s
[CV 4/4] END ......max_depth=3, n_estimators=70;, score=0.544 total time=   0.0s
[CV 1/4] END ......max_depth=3, n_estimators=75;, score=0.469 total time=   0.0s
[CV 2/4] END ......max_depth=3, n_estimators=75;, score=0.654 total time=   0.0s
[CV 3/4] END ......max_depth=3, n_estimators=75;, score=0.564 total time=   0.0s
[CV 4/4] END ......max_depth=3, n_estimators=75;, score=0.569 total time=   0.0s
[CV 1/4] END ......max_depth=3, n_estimators=80;, score=0.479 total time=   0.0s
[CV 2/4] END ......max_depth=3, n_estimators=80;, score=0.654 total time=   0.0s
[CV 3/4] END ......max_depth

[CV 2/4] END ......max_depth=5, n_estimators=85;, score=0.510 total time=   0.1s
[CV 3/4] END ......max_depth=5, n_estimators=85;, score=0.549 total time=   0.1s
[CV 4/4] END ......max_depth=5, n_estimators=85;, score=0.573 total time=   0.1s
[CV 1/4] END ......max_depth=5, n_estimators=90;, score=0.486 total time=   0.1s
[CV 2/4] END ......max_depth=5, n_estimators=90;, score=0.489 total time=   0.1s
[CV 3/4] END ......max_depth=5, n_estimators=90;, score=0.549 total time=   0.1s
[CV 4/4] END ......max_depth=5, n_estimators=90;, score=0.573 total time=   0.1s
[CV 1/4] END ......max_depth=5, n_estimators=95;, score=0.489 total time=   0.1s
[CV 2/4] END ......max_depth=5, n_estimators=95;, score=0.489 total time=   0.1s
[CV 3/4] END ......max_depth=5, n_estimators=95;, score=0.549 total time=   0.1s
[CV 4/4] END ......max_depth=5, n_estimators=95;, score=0.548 total time=   0.1s
[CV 1/4] END .....max_depth=5, n_estimators=100;, score=0.457 total time=   0.1s
[CV 2/4] END .....max_depth=

Добавим данные в таблицу `results`

In [60]:
results.loc[len(results.index)] = ['RandomForestClassifier', best_parameters, f1] 
display(results)

,model,best_parameters_train,F1_CV
0,DecisionTreeClassifier,"{'max_depth': 6, 'min_samples_split': 4}",0.570592
1,RandomForestClassifier,"{'max_depth': 2, 'n_estimators': 95}",0.609714


### Итоги обучения моделей

Методом кросс-валидации с разными гиперпараметрами обучено 2 модели.  
Для каждой модели определены ее оптимальные гиперпараметры и метрика F1.  
Наилучший показатель F1_CV 0.609714 продемонстрировала модель RandomForestClassifier .  
Выберем ее для запуска в продакшен.  
Оценим качество модели на тестовых данных.

## Анализ модели

Создадим функцию:
- Обучения моделей с их оптимальными гиперпараметрами
- Предсказания моделей
- Хронометрирования времени обучения и предсказания

In [61]:
def fit_predict(model, features_train, features_test, target_train, target_test):
    start = time() 
    model.fit(features_train, target_train)
    training_time = time()-start
    
    start = time() 
    predictions = model.predict(features_test)
    predict_time = time()-start
    f1 = f1_score(target_test, predictions, average='micro')
    return training_time, predict_time, f1

### RandomForestClassifier

In [62]:
del final_result

NameError: name 'final_result' is not defined

Оценим качество модели на тестовых данных.

In [ ]:
n = 1
model = RandomForestClassifier(random_state=12345, class_weight='balanced', **results.iloc[n]['best_parameters_train'])
fit_time, predict_time, f1 = fit_predict(model, features_train, features_test, target_train, target_test)
print('\nВремя обучения модели с оптимальными гиперпараметрами    :', fit_time, 'сек.')
print('Время предсказания модели с оптимальными гиперпараметрами:', predict_time, 'сек.')
print('F1 модели:                                              ', f1)
final_result = pd.DataFrame ({'Model':['RandomForestClassifier'], 'fit_time':[fit_time], 'predict_time':[predict_time], 'F1':[f1]})

Проверим модель на адекватность, сравнив качество её предсказаний с качеством модели, которая предсказывала бы константу.  
В качестве константной модели воспользуемся `DummyClassifier`.

### DummyClassifier - most_frequent

In [ ]:
model = DummyClassifier(strategy='most_frequent')
fit_time, predict_time, f1 = fit_predict(model, features_train, features_test, target_train, target_test)
print('\nВремя обучения модели с оптимальными гиперпараметрами    :', fit_time, 'сек.')
print('Время предсказания модели с оптимальными гиперпараметрами:', predict_time, 'сек.')
print('F1 модели:                                              ', f1)
final_result.loc[len(final_result.index)] = ['DummyClassifier - most_frequent', fit_time, predict_time, f1]

### Итоги анализа моделей

In [ ]:
display(final_result) 

Наилучший показатель F1 0.546875 продемонстрировала модель `RandomForestClassifier` .  
Модель адекватна, т.к. ее F1 больше аналичного показателя случайной модели, предсказывающей наиболее частое значение.

## Итоговый вывод

В рамках проекта решалась задача построения модели прогнозирования уровня сложности фильма для изучения английского языка.  
Для разработанной модели предполагается создать web-сервис, загрузив в который субтитры фильма, можно получить метку уровня сложности.  
Для исследования заказчиком была предоставлена база данных, размеченных по уровню сложности субтитров фильмов.
Основной критерием для модели является качество предсказания.

Задача была решена по этапам:

- Подготовка данных
    - Выполнен поиск и дополнительно загружены субтитры фильмов 
    - Исходные данные загружены и исследованы
    - Выполнена очистка текстов, токенезация, лемматизация и удалены стоп-слова
    - Удалены аномальные значения, лишние данные и дубликаты 
    - Тексты субтитров приведены в соответствие со словарями Oxford
    - Созданы дополнительные параметры 
    - Субтитры преобразованы в числовые векторы
    - В данных выделены признаки и целевой признак
- Обучение моделей и подбор оптимальных гиперпараметров
    - DecisionTreeClassifier
    - RandomForestClassifier
- Анализ моделей
    - Проведено обучение наилучшей модели с оптимальными гиперпараметрами
    - Построены ее предсказания
    - Проведен хронометраж времени обучения и предсказания
    - Для оценки качества моделей применялась метрика F1

Наилучшие показатели продемонстрировала модель `RandomForestClassifier` .  
На тестовых данных ее F1 равен 0.546875.

Чтобы повысить качество модели, необходимо увеличить количество размеченных по уровню сложности фильмов.
При этом, в первую очередь, необходимо дополнительно собрать и разметить субтитры для фильмов уровня сложности A1, A2, B1 и C1.
В исходных данных фильмов этих сегментов было критически мало для качественного обучения модели.
Кроме того, следует рассмотреть возможность включения в параметры для обучения моделей иных дополнительных признаков.
Например, среднее кол-во слов в предложениях, скорость речи и т.д.